# Creating a Telegram Bot

* Working library: `python-telegram-bot`
* Bot idea - task tracking:
    1. Creating tasks
    2. Marking tasks as complete
    3. Deleting tasks
    4. Viewing the entire task list

* Information storage: a regular dictionary
    - Alternatively, you can also use a table (Excel, CSV) or a connectable database like PostgreSQL/MySQL, etc.


Let's install the necessary libraries

In [ ]:
# pip install python-telegram-bot

Let's import the necessary modules:

In [ ]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, MessageHandler, Filters, ContextTypes

Let's create an empty dictionary user_tasks, which will be used to store each user's tasks. The key will be the chat ID, and the values will be a list of tasks.

In [ ]:
user_tasks = {}

# Example of what the dictionary will look like when users add task information
{
    'chat_id_1': [
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'}
    'chat_id_2': [
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'}
    ],
    'chat_id_3': [
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'},
        {'Задача': Название задачи №..., 'Статус': 'выполнена/в процессе'}
    ],
    ...
}

Before writing the bot code, it needs to be created on Telegram servers.
The procedure is as follows:

1. Go to @BotFather
2. Click /start, then select /newbot
3. The bot will send a message asking what to name the bot - this name will be displayed in the bot's name
4. After you decide on the bot's name, you need to come up with a username for your bot - this is its nickname (i.e., it usually appears after the @ symbol for users/bots). Important:
* The bot's username must be unique - otherwise, BotFather will not accept it and will inform you about it
* The bot's username must end in bot/_bot/BOT
* The bot's username should be understandable: if we are making a bot about tasks, it could be something like task_manager_bot
5. Finally, BotFather will send you a message about the successful creation of your bot. In the message, it is important to copy your bot's TOKEN - it will be used to make requests to the TG server via the API

## Writing the Bot Code

### Handler of /start command

The first thing to implement when creating a bot is what to do when the user clicks the /start button.

Let's define the **asynchronous** function start, which is launched when the user calls the `/start` command.


#### mportant!

All functions that imply direct interaction with the Telegram chat must be asynchronous - this way, the bot will be able to concurrently process requests from many users simultaneously (for example, I pressed the `/start` button, and at the same time someone else sent a text message to the bot in response to some question)


Each function implying interaction with the chat has two parameters: `update` and `context`. Details:
* `update` - contains information about the received update from Telegram. It can include various data, such as messages, commands, button presses, etc.

   
**Contents**: Depending on the type of update, the  `update` object may contain:
- `update.message`: The message that was sent (if it's a text message).
- `update.callback_query`: Data about a button press (if an inline button is used).
- `update.chat`: Information about the chat.
- `update.user`: Information about the user who sent the message.
    
* `context` - provides additional context for the handler (bot). It is used to pass information between handlers and may contain state or data that may be useful during processing.
    
**Contents**: The `context` bject may include:
- `context.bot`: A reference to your bot instance, allowing you to send messages and perform other actions.
- `context.args`: A list of arguments passed to the command.
- `context.user_data`: A dictionary for storing user-specific data (e.g., temporary data that you want to save between calls).
- `context.chat_data`: A dictionary for storing chat-specific data.
- `context.bot_data`: A dictionary for storing bot-specific data.

Let's implement the **functionality** of what we want to do when the user clicks the  `/start` button:
1. Let's take its chat_id (**not to be confused with the user's username**, chat_id - is a unique chat identifier)
2. If the user is not in our general dictionary with all the information, add the chat ID as the key and an empty list as the value

In [ ]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_chat_id = update.message.chat.id
    if user_chat_id not in user_tasks:
        user_tasks[user_chat_id] = []

When replying to the user, let's add a keyboard that we will attach to the text message. To do this, let's create a list of buttons, where it is important to specify two parameters:

1. Button text (what will be written on the button)
2. Button `callback_data`

#### Important!
`callback_data` - is a unique identifier for each button; it must not be repeated, otherwise, the bot will not understand which specific button was pressed

Features of creating keyboards:
* If you pay attention, then in general, the keyboard looks like a list of lists, where the inner list shows how many buttons will be **in EACH row**
* If you want to create several buttons **in ONE row**, for this, in the list of the corresponding row, you need to pass several buttons. For example:

`[
    
    [InlineKeyboardButton("Button 1", callback_data='data_1'), InlineKeyboardButton("Button 2", callback_data='data_2')]
]
`


In our case, let's make 4 rows of 1 button each with the corresponding text:

In [ ]:
keyboard = [
    [InlineKeyboardButton("All tasks", callback_data='all_tasks')],
    [InlineKeyboardButton("Add task", callback_data='add_task')],
    [InlineKeyboardButton("Delete task", callback_data='delete_task')],
    [InlineKeyboardButton("Complete task", callback_data='complete_task')],
]


After creating the keyboard skeleton/image, it is necessary to apply the `InlineKeyboardMarkup` function, which will turn the list of buttons into a full-fledged keyboard image so that Telegram understands us:

In [ ]:
markup = InlineKeyboardMarkup(keyboard)

After all the preparatory work is done, we are ready to send a message to the user in response to their pressing the`/start`button. To send a message **to the same** chat, the `reply_text` function of the `update.message` object is used. The required parameter is `text` - the text of your message  :)

Additional parameters:
* `reply_markup` - the parameter responsible for the keyboard. We will pass our created keyboard variable here.
* `parse_mode` - the method for styling your text. `Markdown` or `MarkdownV2` is most often used. You can read more about text styling [here](https://habr.com/ru/sandbox/170069/)

#### Important!

Don't forget that sending a message is also an asynchronous action, so you need to put `await` before sending.

In [ ]:
await update.message.reply_text(
    text="Hello! I am your personal task manager.\nChoose an action:",
    reply_markup=markup
)

Since we used an `Inline` keyboard, we need to write a function that will understand what to do when a corresponding button is pressed.

Let's create a function `button_handler`, which also takes `update` and `context` as input.



In [ ]:
async def button_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    # some futher code

### Button handler

All information about which button was pressed is stored in the `update.callback_query`object, so to avoid carrying it everywhere, let's save it to a separate variable `query`.

Next, we **must** write `await query.answer()`, thus letting Telegram know that the bot received information about the button and that Telegram doesn't need to show a notification about an unconfirmed response.


In [ ]:

    query = update.callback_query
    await query.answer()

By analogy with the `start` function, we collect the necessary information about the user, but now from the `query` object. Namely, we take the user identifier - `chat_id`.



In [ ]:
user_chat_id = query.from_user.id

Remember that when creating the keyboard, we created the `callback_data` parameter - with its help we will get information about what to do depending on which button was pressed. This is stored in the `query.data` attribute.

Let's write the functionality of processing what to do depending on the "data" of each button:


In [ ]:
if query.data == 'all_tasks':
    # do something if the add_task button was selected
    # for example await show_tasks(query, user_chat_id)

If the "Add task" button was pressed, we edit the message to request the text of the new task, and save the action in `context.user_data` to know that the user is going to add a task.

In [ ]:
elif query.data == 'add_task':
    # do something if the add_task button was selected
    # for example await query.edit_message_text("Enter the task text:")
    # context.user_data['action'] = 'add'

In [ ]:
elif query.data == 'delete_task':
    # do something if the delete_task button was selected
    # for example await show_tasks(query, context, user_chat_id, delete=True)

If the "Complete task" button is pressed, we call `show_tasks`, passing the `complete=True` parameter to show tasks for completion.


In [ ]:
elif query.data == 'complete_task':
    # do something if the complete_task button was selected
    # for example await show_tasks(query, context, user_chat_id, complete=True)

Let's consider the case where the user pressed the button with the `all_tasks` information:

In [ ]:
if query.data == 'all_tasks':
    await show_tasks(query, context, user_chat_id)

In this case, we tell the bot that if a button with the `all_tasks` data is pressed, go to the `show_tasks` function, to which we pass as parameters:

* query
* context
* user_chat_id

Looking ahead, I will say that this function will send a message to the user, so before calling the function you need to put `await`. If the function did not involve any interaction with the chat and was only auxiliary, `await` would not be necessary.


**Function for displaying tasks**

#### Important

Since Python goes FROM TOP TO BOTTOM, functions must be stored in its memory in this order. That is, first you define a function, then somewhere later in the code you call it. Thus, the structure of our code (so far) should look like this:


1. import libraries
2. creation of global variables (token/dictionary/variables)
3. all working/auxiliary functions
4. function for handling the `/start` command
5. function for handling button presses


What will our `show_tasks`function consist of?

First, if we don't pass the `delete` and `complete` parameters (which are responsible for the flag that we want to delete/complete the task, respectively), our function will simply output a single message with all tasks in the format:

Task list in progress:

1. Task 1
2. Task 2

Completed task list:

1. Task 3
2. Task 4



In [ ]:
async def show_tasks(update: Update, context, user_chat_id: int, delete=False, complete=False) -> None:

We check if the user has any tasks. If not, we send a message and exit the function.


In [ ]:
if len(user_tasks[user_chat_id]) == 0:
    await update.message.reply_text("У вас еще нет задач.")
    return ConversationHandler.END # Forced conversation termination / chat end

If the user has tasks, the `if` condition won't be met, and we proceed to the next part of the code where we get the user's task list.

We create two lists: one for tasks with the status "in progress", the other for completed tasks.


In [ ]:
tasks = user_tasks[user_chat_id]
in_progress = [task for task in tasks if task['status'] == 'In progress']
completed = [task for task in tasks if task['status'] == 'Completed']

When we called the `show_tasks` function, we didn't pass the `delete` and `complete` parameters, so the following `if` conditions won't be met, and we'll "jump" to the bottom of the function.

However, if the `delete=True` parameter is specified, we create a message that displays a list of all tasks `in progress`, so the user can choose which task they want to delete (= remove from the list in the dictionary):

1. We form a string with the numbers and text of all tasks that are in progress. If there are no tasks, we show a corresponding message.
2. We ask the user for the task number to delete and save the action in `context.user_data`.


In [ ]:
if delete:
    tasks_list = "\n".join([f"{i + 1}: {task['text']}" for i, task in enumerate(in_progress)])
    await update.message.reply_text(f"Task list:\n{tasks_list if tasks_list else 'No tasks to delete.'}")

    await update.message.reply_text("Select the task number to delete:")
    context.user_data['action'] = 'delete'

If `complete=True`, we display the tasks to be completed.

Similarly, we create a string with tasks to be completed.

We request the task number to complete and save the action in `context.user_data`.



In [ ]:
elif complete:
    tasks_list = "\n".join([f"{i + 1}: {task['text']}" for i, task in enumerate(in_progress)])
    await update.message.reply_text(f"List of tasks in progress:\n{tasks_list if tasks_list else 'No tasks.'}")

    await update.message.reply_text("Select the task number to complete:")
    context.user_data['action'] = 'complete'

If none of the conditions were met, we display the current tasks by status.

In [ ]:
else:
    tasks_in_progress = "\n".join([f"{i + 1}: {task['text']}" for i, task in enumerate(in_progress)])
    tasks_completed = "\n".join([f"{i + 1}: {task['text']}" for i, task in enumerate(completed)])

    await update.message.reply_text(f"Tasks in progress:\n{tasks_in_progress if tasks_in_progress else 'No tasks.'}\n\nCompleted tasks:\n{tasks_completed if tasks_completed else 'No tasks.'}")

I recommend trying to create a test dictionary in Jupyter and carefully study **step-by-step** how the functions used above work.







Ultimately, our function should look like this:

In [ ]:
async def show_tasks(update, context, user_chat_id: int, delete=False, complete=False) -> None:
    if user_chat_id not in user_tasks:
        await update.message.reply_text("You have no tasks yet.")
        return ConversationHandler.END

    tasks = user_tasks[user_chat_id]
    in_progress = [task for task in tasks if task['status'] == 'In progress']
    completed = [task for task in tasks if task['status'] == 'Completed']

    if delete:
        tasks_list = "\n".join([f"{i + 1}: {task['text']}" for i, task in enumerate(in_progress)])
        await update.message.reply_text(f"Task list:\n{tasks_list if tasks_list else 'No tasks to delete.'}")

        await update.message.reply_text("Select the task number to delete:")
        context.user_data['action'] = 'delete'

    elif complete:
        tasks_list = "\n".join([f"{i + 1}: {task['text']}" for i, task in enumerate(in_progress)])
         await update.message.reply_text(f"Tasks in progress:\n{tasks_list if tasks_list else 'No tasks.'}")

        await update.message.reply_text("Select the task number to complete:")
        context.user_data['action'] = 'complete'
    else:
        tasks_in_progress = "\n".join([f"{i + 1}: {task['text']}" for i, task in enumerate(in_progress)])
        tasks_completed = "\n".join([f"{i + 1}: {task['text']}" for i, task in enumerate(completed)])

       await update.message.reply_text(
            text=f"Tasks in progress:\n{tasks_in_progress if tasks_in_progress else 'No tasks.'}\n\nCompleted tasks:\n{tasks_completed if tasks_completed else 'No tasks.'}"
      )

#### Important!
Note: inside the `delete` and `complete` cases, the code `context.user_data['action'] = ...` is used.


Reminder: the `context` parameter can store **global** data with information about the user. `Context.user_data` is essentially the simplest dictionary. Thus, to store information throughout the user's interaction with the bot about what action the person performed, we created the `action` key, into the value of which we wrote `delete` and `complete` depending on what the user did (we will need this later).


---

Now let's discuss the case where the bot sends the user a message implying a keyboard response from the person, for example:
* The bot asked to enter the task name.
* The bot asked to enter the task number to delete.
* The bot asked to enter the task number to mark as completed.

The key idea is that the response to the bot's message will be **text input from the keyboard**.



**Message Handling**

Let's define an asynchronous function `handle_message` that handles text messages from the user.

As usual, the function receives two parameters as input: `update` and `context`. We collect the chat ID and the current user action (this is where we need the use of `context.user_data`, which we defined earlier).



In [ ]:
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_chat_id = update.effective_chat.id
    action = context.user_data.get('action')


If the current task is adding, we get the task text from the `update.message.text` object.

Next, we add a **sub**dictionary to the user's task list (i.e., we access it by key - chat_id) in the format:

`{'text': task name, 'status': task status}`

* By default, we will assume that when a task is added, its status is **"in progress".**
Then we send a message that the task has been added and reset the action.



In [ ]:
if action == 'add':
    task_text = update.message.text
    user_tasks[user_chat_id].append({'text': task_text, 'status': 'in progress'})
    await update.message.reply_text("Task added.")
    context.user_data['action'] = None

**By analogy:**

If the action is deletion, we get the task index from the message text (keeping in mind that the user, when answering the question, specified the number = task index), check its correctness, and delete it if it is valid.

We send the corresponding message and reset the action.



In [ ]:
elif action == 'delete':
        task_index = int(update.message.text) - 1
        if 0 <= task_index < len(user_tasks[user_chat_id]):
            user_tasks[user_chat_id].pop(task_index)
            await update.message.reply_text("Task deleted.")
        else:
            await update.message.reply_text("Incorrect task number.")
        context.user_data['action'] = None

If the action is task completion, similarly, we get the index, check it, and change the task status. We send a message about completion.

In [ ]:
elif action == 'complete':
        task_index = int(update.message.text) - 1
        if 0 <= task_index < len(user_tasks[user_chat_id]):
            user_tasks[user_chat_id][task_index]['status'] = 'completed'
            await update.message.reply_text("Task completed.")
        else:
            await update.message.reply_text("Incorrect task number.")
        context.user_data['action'] = None

## Main Function

After we have written ALL the scenarios we want to consider, the last and main step remains - **to put everything together**. In other words, we

1. Assemble the bot image.
2. Pass information about which functions need to be run in which scenarios (triggers).
3. Write the bot launch function.


We define **the main asynchronous** function (usually `main`) - nothing needs to be passed into it.

Next, we create an application instance using the bot token.


In [ ]:
def main() -> None:
    app = Application.builder().token(TOKEN).build()

**Scenarios**


Scenarios, or  **handlers** , are a way to give the bot information about which function to run in which cases. Namely, we remember that we have three types of interaction with the program:

1. Pressing commands (`/start`, `/help`, `/rules`, etc.)
    * **Important** - a separate function must be written for each command.
2. Pressing `inline` buttons
3. Sending text


**More details:**:

* To tell the bot that when the `/start` command is sent, the `start` function should be launched, the `CommandHandler("start", start)` construct is used, where:
  * The first parameter specifies the command name.
  * The second parameter specifies the function to be called.


* To tell the bot that the `button_handler` function should be called when buttons are pressed, the `CallbackQueryHandler(button_handler)` construct is used.


**Important:**
    
* If you understand that you have only ONE function to handle ALL buttons, it is sufficient to specify the name of the function that will be launched when a button is pressed.
* If you want to separate functions (i.e., conditionally, when buttons 1, 2 are pressed, function 1 will work, and when buttons 3, 4 are pressed - another), you can use regular expressions to directly specify which function to launch for the corresponding button. Example:


`CallbackQueryHandler(button_handler_1, pattern='^(button_1|button_2$')`

`CallbackQueryHandler(button_handler_2, pattern='^(button_3|button_4$')`

In this case, if any of the buttons with `callback_data` = **button_1** or **button_2** is pressed, the `button_handler_1` function will be executed, otherwise - `button_handler_2`

* To tell the bot that the `handle_message` function should be called when a text message is sent to the chat, the `MessageHandler(Filters.text & ~Filters.command, handle_message)`construct is used, where:

    * `Filters.text & ~Filters.command` acts as a trigger indicating that text has been sent.

    * `handle_message` is the function to be launched.


**Important**

Similar to `CallbackQueryHandler`, text "patterns" can be customized using regular expressions by specifying the corresponding value of the `pattern`. parameter. However, this only works if you know the format of keyboard input for sure.

After all three triggers have been defined, they need to be added to the bot's memory one by one using the `.add_handler(...)`  function.


In [ ]:
app.add_handler(CommandHandler("start", start))
app.add_handler(CallbackQueryHandler(button_handler))
app.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

Finally, after adding all the "handlers", it only remains to tell the program that it is necessary to run the bot continuously:


In [ ]:
app.run_polling()

## Launch

The final step is to call the function that will generate the bot, create all the scenarios, and launch our bot!

In [ ]:
if __name__ == '__main__':
    main()

## Further Improvements

What we covered today is just the tip of the iceberg in terms of creating functionality for Telegram bots. In addition to all this, bots allow you to:

* Send files (e.g., tables) to the chat.
* Receive files sent by the user from the chat.
* Send/download images.
* Interact with databases.
* Perform scheduled actions using the `scheduler` library.
* Perform actions according to a specific scenario (e.g., if button 1 was pressed first, then button 2, then special input from the user, etc.).


#### Important

Remember that when you write code on your computer, it acts as your "host," the core of the computing power. As soon as you turn off your computer, the bot stops working. To fix this, programmers rent virtual machines to which they move their code so that it runs constantly. However, this is a separate topic and a completely different story!


Don't be afraid to learn about Telegram bots to simplify and automate tasks you're interested in!

And don't forget that bots have limitations - [more details](https://www.unisender.com/ru/blog/kakie-limity-est-v-telegram/)